In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
import json

permission = "data_ai"

# 파일 경로
file_path = f"qwen3_company_train_dataset_{permission}2.json"

tool_prompt = f"사용자는 Data AI(데이터 AI)팀에 속한 팀원입니다. 질문이 들어오면 반드시 {permission}_search 툴을 호출하여 문서를 검색하세요."

new_system_message = f"""
당신은 사내 지식을 활용하여 사용자의 질문에 정확하고 유용한 답변을 제공하는 한국인 AI 비서입니다.
다음 지침을 따르세요:
1. 기존의 말투는 잊고 가볍고 친근한 반말로 답변하세요.
2. 대화 내역의 말투도 참고하지 말고 무조건 가볍고 친근한 반말로 답변하세요.
3. 사실에 기반한 정보를 사용하세요.
4. 사용자의 질문에 대한 답변을 문서에서 찾을 수 없을 경우, "잘 모르겠어"라고 솔직하게 말하세요.
5. 사용자가 문서에 대한 질문이 아닌, "안녕"과 같은 일상적인 질문을 한다면 해당 내용에 대해서 적절히 답변해주세요.
6. 답변이 너무 길지 않게 하세요.
7. 사용자가 정중하고 사무적인 어조로 물어보더라도, 반드시 가볍고 친근한 반말로 답변로 답변해야 합니다.
8. {tool_prompt}
"""

# JSON 파일 열기
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 각 대화별로 첫 번째 system 메시지 수정
for conversation in data:
    messages = conversation.get("messages", [])
    if messages and messages[0]["role"] == "system":
        messages[0]["content"] = new_system_message

new_file_path = f"qwen3_company_train_dataset_{permission}_informal2.json"

# 수정된 JSON 파일 저장
with open(new_file_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("시스템 프롬프트가 모두 수정되었습니다.")


시스템 프롬프트가 모두 수정되었습니다.


In [3]:
import json
from openai import OpenAI
from tqdm import tqdm

client = OpenAI()

# JSON 로드
with open(new_file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

for conversation in tqdm(data):
    for msg in conversation.get("messages", []):

        if msg["role"] == "assistant" and not msg["content"].startswith("<tool_call>"):
            original_text = msg["content"]

            # GPT를 이용해 반말체로 변환
            response = client.chat.completions.create(
                model="gpt-4o-mini",  # 원하는 모델로 변경 가능
                messages=[
                    {"role": "system", "content": "너는 문장을 반말체로 바꿔주는 변환기야. 내용 누락하지 말고 없는 내용도 지어내지 말고 존댓말을 모두 반말로 바꿔줘."},
                    {"role": "user", "content": original_text},
                ],
                temperature=0.2 # 조금의 말투 변환 창의성을 위해서 0.2 정도로만 설정
            )

            informal_text = response.choices[0].message.content.strip()
            msg["content"] = informal_text
            # print(informal_text)

# 새로운 JSON 파일 저장
with open(new_file_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f"assistant 답변을 반말체로 변환해서 '{new_file_path}'에 저장 완료.")


 83%|████████▎ | 275/330 [20:29<04:05,  4.47s/it]


KeyboardInterrupt: 